# Learning
This notebook described how to use Neo4j and SageMaker together.  It takes Neo4j graph embedding data from S3 and trains a supervised learning model using SageMaker.

Click this button to open the notebook in SageMaker Studio Lab. [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/neo4j-partners/neo4j-sagemaker/blob/main/form13.ipynb)

## Setup AWS Connection
We're going to setup a boto3 connection.  It seems like Studiolab might have its own cred system we should use instead.

In [ ]:
%pip install boto3

In [ ]:
# Edit these variables!
ACCESS_KEY = 'your-access-key'
SECRET_KEY = 'your-secret-key'

In [ ]:
import boto3

file_name='embedding.csv'
object_name=file_name
bucket_name = ACCESS_KEY.lower() + '-form13'

s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY)
s3.download_file(bucket_name, object_name, file_name)

## Train SageMaker Model
To do --- train a SageMaker model on embedding.csv